In [ ]:
import numpy as np
import pandas as pd
import optuna
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, log_idf_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import train_test_split, ranking_metrics_at_k

import cornac


In [2]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [3]:
movielens_df = (
    pd.DataFrame(
        data=cornac.datasets.movielens.load_feedback(variant="20M"),
        columns=['user_id', 'item_id', 'target']
    )
    .loc[:, ['user_id', 'item_id', 'target']]
    .dropna()
)
movielens_df['user_id'].nunique(), movielens_df['item_id'].nunique(), movielens_df.shape[0]

Data from http://files.grouplens.org/datasets/movielens/ml-20m.zip
will be cached into /home/coder/.cornac/ml-20m/ratings.csv


0.00B [00:00, ?B/s]

Unzipping ...
File cached!


(138493, 26744, 20000263)

In [4]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    movielens_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (138493, 26744), Val Shape: (138493, 26744), Test Shape: (138493, 26744)


In [ ]:
results_folder = "results/movielens_20m_lmf"
results_filename = "movielens_20m_lmf_results.csv"

import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    algorithms = {
        "LMF_factors=100": lambda: LogisticMatrixFactorization(
            factors=100,
            learning_rate=1,
            regularization=0.6,
            iterations=30,
            random_state=42
        ),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "log_idf
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "log_idf
            weighted = log_idf_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "log_idf
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            params["p"] = trial.suggest_float("p", 0.1, 1.5)
        weighted_train = get_weighted_matrix(train_mat, params)

        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ranking_metrics_at_k(model, train_mat, val_mat, K=20, show_progress=False)['ndcg']

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "log_idf"power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    metrics_at_10 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    metrics_at_20 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": metrics_at_10['ndcg'],
        "Test NDCG@20": metrics_at_20['ndcg'],
        "Test Precision@10": metrics_at_10['precision'],
        "Test Precision@20": metrics_at_20['precision'],
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [6]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [ ]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:49:58,653] A new study created in memory with name: no-name-fdccf3ce-5e75-4c2c-b366-28b02c2b648f


Running optimization for LMF_factors=100 with no_weighting...


[I 2026-02-08 11:52:04,852] Trial 0 finished with value: 0.03623736047903792 and parameters: {}. Best is trial 0 with value: 0.03623736047903792.


In [ ]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:47:23,356] A new study created in memory with name: no-name-66bec88d-ce95-4373-9952-9519ac199340


Running optimization for LMF_factors=100 with bm25...


[I 2026-02-08 11:47:28,270] Trial 4 finished with value: 0.11093201450111392 and parameters: {'bm25_k1': 449.5435146186711, 'bm25_b': 0.9673521867428252}. Best is trial 4 with value: 0.11093201450111392.
[I 2026-02-08 11:47:28,431] Trial 7 finished with value: 0.11331301948421049 and parameters: {'bm25_k1': 252.49082695516736, 'bm25_b': 0.9140693061206121}. Best is trial 7 with value: 0.11331301948421049.
[I 2026-02-08 11:47:28,441] Trial 6 finished with value: 0.11399842397323168 and parameters: {'bm25_k1': 225.27138083941168, 'bm25_b': 0.799687653452424}. Best is trial 6 with value: 0.11399842397323168.
[I 2026-02-08 11:47:28,462] Trial 1 finished with value: 0.11862926360337808 and parameters: {'bm25_k1': 347.3224725671412, 'bm25_b': 0.7496673602128733}. Best is trial 1 with value: 0.11862926360337808.
[I 2026-02-08 11:47:28,494] Trial 2 finished with value: 0.12426670191812765 and parameters: {'bm25_k1': 890.9368293378736, 'bm25_b': 0.062470166850947195}. Best is trial 2 with value

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,bm25,20,0.124267,0.122301,0.146816,0.13838,0.17847,0.606669,"{'bm25_k1': 890.9368293378736, 'bm25_b': 0.062..."


In [ ]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)


[I 2026-02-08 11:47:37,753] A new study created in memory with name: no-name-d54e0ccb-cb1e-4ee2-9f30-be538a4783b6


Running optimization for LMF_factors=100 with tfidf...


[I 2026-02-08 11:47:38,455] Trial 0 finished with value: 0.12357519240498764 and parameters: {}. Best is trial 0 with value: 0.12357519240498764.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,tfidf,1,0.123575,0.127691,0.149315,0.144318,0.180602,0.627758,{}


In [ ]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:47:39,922] A new study created in memory with name: no-name-585adf99-f04b-44f3-b2db-ca2425d33c57


Running optimization for LMF_factors=100 with log...


[I 2026-02-08 11:47:40,658] Trial 0 finished with value: 0.1303761409841096 and parameters: {}. Best is trial 0 with value: 0.1303761409841096.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,log,1,0.130376,0.137035,0.152675,0.150091,0.173614,0.599544,{}


In [ ]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log_idf", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:47:42,058] A new study created in memory with name: no-name-384feb5d-eb73-4e55-93f3-7c187d187e62


Running optimization for LMF_factors=100 with confidence...


[I 2026-02-08 11:47:47,066] Trial 7 finished with value: 0.02846572732681722 and parameters: {'conf_alpha': 76.385882746759}. Best is trial 7 with value: 0.02846572732681722.
[I 2026-02-08 11:47:47,114] Trial 3 finished with value: 0.030005729328642622 and parameters: {'conf_alpha': 105.95305072498014}. Best is trial 3 with value: 0.030005729328642622.
[I 2026-02-08 11:47:47,138] Trial 0 finished with value: 0.07880715451231446 and parameters: {'conf_alpha': 8.240475452849108}. Best is trial 0 with value: 0.07880715451231446.
[I 2026-02-08 11:47:47,148] Trial 6 finished with value: 0.02976503091219951 and parameters: {'conf_alpha': 122.3952391845651}. Best is trial 0 with value: 0.07880715451231446.
[I 2026-02-08 11:47:47,150] Trial 2 finished with value: 0.030534501193680872 and parameters: {'conf_alpha': 135.6859830832092}. Best is trial 0 with value: 0.07880715451231446.
[I 2026-02-08 11:47:47,182] Trial 1 finished with value: 0.03022415108429704 and parameters: {'conf_alpha': 138.5

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,confidence,20,0.11609,0.117568,0.139725,0.138215,0.174917,0.579642,{'conf_alpha': 1.3997319432813828}


In [ ]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:47:56,627] A new study created in memory with name: no-name-bd7bfe21-eae5-4afe-9fc7-5d90e6660970


Running optimization for LMF_factors=100 with power...


[I 2026-02-08 11:48:01,586] Trial 0 finished with value: 0.10698715095841836 and parameters: {'power_p': 1.0515309700044204}. Best is trial 0 with value: 0.10698715095841836.
[I 2026-02-08 11:48:01,622] Trial 4 finished with value: 0.10961850801790703 and parameters: {'power_p': 1.1767693616659802}. Best is trial 4 with value: 0.10961850801790703.
[I 2026-02-08 11:48:01,659] Trial 6 finished with value: 0.1135958824711078 and parameters: {'power_p': 0.709907615432767}. Best is trial 6 with value: 0.1135958824711078.
[I 2026-02-08 11:48:01,690] Trial 7 finished with value: 0.13189736524836917 and parameters: {'power_p': 0.1427936776274828}. Best is trial 7 with value: 0.13189736524836917.
[I 2026-02-08 11:48:01,693] Trial 2 finished with value: 0.12692175865201785 and parameters: {'power_p': 0.18753767196098947}. Best is trial 7 with value: 0.13189736524836917.
[I 2026-02-08 11:48:01,720] Trial 1 finished with value: 0.11666015336075071 and parameters: {'power_p': 1.2218118643528066}. B

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,power,20,0.136998,0.142695,0.159872,0.150256,0.171127,0.641469,{'power_p': 0.12019760331996314}


In [ ]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:48:11,154] A new study created in memory with name: no-name-220f1ff3-b2a0-4031-b166-9450f4b27b86


Running optimization for LMF_factors=100 with normalized...


[I 2026-02-08 11:48:11,871] Trial 0 finished with value: 0.14560554872075088 and parameters: {}. Best is trial 0 with value: 0.14560554872075088.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,normalized,1,0.145606,0.156788,0.171044,0.159327,0.178233,0.60226,{}


In [ ]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:48:13,381] A new study created in memory with name: no-name-979fc17f-ed3c-41b8-82e6-d6fe245e972f


Running optimization for LMF_factors=100 with pmi...


[I 2026-02-08 11:48:14,102] Trial 0 finished with value: 0.11136221638197882 and parameters: {}. Best is trial 0 with value: 0.11136221638197882.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,pmi,1,0.111362,0.11663,0.138991,0.130134,0.164969,0.596271,{}


In [ ]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:48:15,539] A new study created in memory with name: no-name-0be3a974-ec33-41d2-984f-df9c7163d4e8


Running optimization for LMF_factors=100 with robust_user_centric...


In [ ]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:44:51,078] A new study created in memory with name: no-name-2ab10bd6-f77f-40e0-80ce-a3732164fa40


Running optimization for LMF_factors=100 with robust_user_centric_weight_v2...


/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:364: RuntimeWarning: overflow encountered in exp
  weights = 1 / (1 + np.exp(-z_scores))
[I 2026-02-08 11:44:55,959] Trial 1 finished with value: 0.14222767274866804 and parameters: {'lower_q': 44.946864213449814, 'upper_q': 86.61588375369048}. Best is trial 1 with value: 0.14222767274866804.
[I 2026-02-08 11:44:56,385] Trial 2 finished with value: 0.13714585698794177 and parameters: {'lower_q': 18.42790712620242, 'upper_q': 55.608139480947784}. Best is trial 1 with value: 0.14222767274866804.
[I 2026-02-08 11:44:56,390] Trial 5 finished with value: 0.13742459371698043 and parameters: {'lower_q': 31.8209239006363, 'upper_q': 74.36206492146295}. Best is trial 1 with value: 0.14222767274866804.
[I 2026-02-08 11:44:56,405] Trial 0 finished with value: 0.1294827998839493 and parameters: {'lower_q': 9.037524893785452, 'upper_q': 87.33939078156428}. Best is trial 1 with value: 0.14222767274866804.
[I 2026-02-08 11

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,robust_user_centric_weight_v2,20,0.142228,0.143141,0.165339,0.150586,0.185457,0.621233,"{'lower_q': 44.946864213449814, 'upper_q': 86...."


In [ ]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:45:06,157] A new study created in memory with name: no-name-a551056e-259c-4bb9-b430-29f52764ed31


Running optimization for LMF_factors=100 with sigmoid_propensity...


[I 2026-02-08 11:45:11,156] Trial 3 finished with value: 0.12374954766339684 and parameters: {'p': 1.4509957613294016, 'beta': 0.6585785651836707}. Best is trial 3 with value: 0.12374954766339684.
[I 2026-02-08 11:45:11,173] Trial 5 finished with value: 0.1197167948838753 and parameters: {'p': 3.7097370057132393, 'beta': 0.34682747928504254}. Best is trial 3 with value: 0.12374954766339684.
[I 2026-02-08 11:45:11,183] Trial 0 finished with value: 0.12920347514101319 and parameters: {'p': 0.5156936594033215, 'beta': 0.3670299134282563}. Best is trial 0 with value: 0.12920347514101319.
[I 2026-02-08 11:45:11,259] Trial 1 finished with value: 0.12421977546127613 and parameters: {'p': 2.0757913108950627, 'beta': 0.11085473683532743}. Best is trial 0 with value: 0.12920347514101319.
[I 2026-02-08 11:45:11,263] Trial 7 finished with value: 0.12160269850097663 and parameters: {'p': 1.3941599512777827, 'beta': 0.6216201433855186}. Best is trial 0 with value: 0.12920347514101319.
[I 2026-02-08 

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,sigmoid_propensity,20,0.14003,0.145148,0.165106,0.158337,0.183207,0.599144,"{'p': 1.584174552551136, 'beta': 0.82173319199..."


In [ ]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="LMF_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 11:45:20,407] A new study created in memory with name: no-name-4b456f55-39ac-4203-8445-135efda06605


Running optimization for LMF_factors=100 with power_lift...


[I 2026-02-08 11:45:25,479] Trial 7 finished with value: 0.12665975911493066 and parameters: {'p': 0.18733368672913622}. Best is trial 7 with value: 0.12665975911493066.
[I 2026-02-08 11:45:25,548] Trial 6 finished with value: 0.018375867058276235 and parameters: {'p': 1.0853421665358742}. Best is trial 7 with value: 0.12665975911493066.
[I 2026-02-08 11:45:25,569] Trial 2 finished with value: 0.11536278852377933 and parameters: {'p': 0.752828961047047}. Best is trial 7 with value: 0.12665975911493066.
[I 2026-02-08 11:45:25,604] Trial 3 finished with value: 0.1066529097725081 and parameters: {'p': 0.7711447584390401}. Best is trial 7 with value: 0.12665975911493066.
[I 2026-02-08 11:45:25,613] Trial 4 finished with value: 0.11707464605690672 and parameters: {'p': 0.12838678409424592}. Best is trial 7 with value: 0.12665975911493066.
[I 2026-02-08 11:45:25,634] Trial 5 finished with value: 0.11583752714337903 and parameters: {'p': 0.11489324855895683}. Best is trial 7 with value: 0.126

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,power_lift,20,0.12666,0.138709,0.157831,0.146462,0.169588,0.630358,{'p': 0.18733368672913622}


In [ ]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,LMF_factors=100,normalized,1,0.145606,0.156788,0.171044,0.159327,0.178233,0.580045,{}
0,LMF_factors=100,power,20,0.137803,0.153017,0.169674,0.157843,0.176930,0.631229,{'power_p': 0.12873544599079584}
0,LMF_factors=100,robust_user_centric_weight_v2,20,0.142228,0.143141,0.165339,0.150586,0.185457,0.621233,"{'lower_q': 44.946864213449814, 'upper_q': 86...."
0,LMF_factors=100,sigmoid_propensity,20,0.140030,0.145148,0.165106,0.158337,0.183207,0.599144,"{'p': 1.584174552551136, 'beta': 0.82173319199..."
0,LMF_factors=100,robust_user_centric,20,0.145370,0.136959,0.161276,0.142009,0.178707,0.675411,{'scale_factor': 0.3177260922068834}
0,LMF_factors=100,power_lift,20,0.126660,0.138709,0.157831,0.146462,0.169588,0.630358,{'p': 0.18733368672913622}
0,LMF_factors=100,log,1,0.130376,0.137035,0.152675,0.150091,0.173614,0.594178,{}
0,LMF_factors=100,tfidf,1,0.123575,0.127691,0.149315,0.144318,0.180602,0.569084,{}
0,LMF_factors=100,bm25,20,0.124334,0.125421,0.146730,0.139865,0.175865,0.609846,"{'bm25_k1': 970.481333911463, 'bm25_b': 0.0969..."
0,LMF_factors=100,pmi,1,0.111362,0.116630,0.138991,0.130134,0.164969,0.607751,{}
